# BASIC INFORMATION

1.   Date of code written: April 2020
2.   Last Modified Date: N/A
3.   Document Written Date: May 2021

https://www.sungwooshon.tech/project/but-what-skills

## Import File

Dataset: Data Science Job Posting on Glassdoor (https://www.kaggle.com/rashikrahmanpritom/data-science-job-posting-on-glassdoor)

In [18]:
import pandas as pd
data='https://raw.githubusercontent.com/ohhappylife/searchJobs/main/DS_jobs.csv?token=AQICGTYLCE56N25NX7I7DYLAU43EC'

df=pd.read_csv(data, encoding = "ISO-8859-1")

## Text Clearation Remove the Stopword, Tokenization

In [19]:
from nltk.corpus import stopwords
import nltk
stop = stopwords.words('english')

# Removing Stopwords from the list
df['Job Description'] = df['Job Description'].astype(str).str.replace("[^\w\s]", "").str.lower()
df['Job Description'] = df['Job Description'].apply(lambda x: [item for item in x.split() if item not in stop])

In [20]:
# Keep unique values only
df['Job Description'] = df['Job Description'].astype(str).str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [21]:
# Formatting texts
df['Job Description']=df['Job Description'].str.replace("'", '')
df['Job Description']=df['Job Description'].str.replace(",", ' ')
df['Salary Estimate']=df['Salary Estimate'].str.replace("$", '')
df['Salary Estimate']=df['Salary Estimate'].str.replace("K", '')
df['Salary Estimate']=df['Salary Estimate'].str.replace("((Glassdoor est.))", '')

# Find the industrial demand skills

## Count the Tokenized words in job description
List the words in the job descriptions and get the frequency of the words.

In [22]:
# Count the word frequencies
count = df['Job Description'].astype(str).str.split(expand=True).stack().value_counts()

In [23]:
# Store the word frequency
df2 = pd.DataFrame(data = count)
df2.columns = ["count"]

In [24]:
# see the list of the words and their frequency
df2

,count
data,659
experience,647
work,550
science,535
skills,532
...,...
histopathology,1
themes,1
8vc,1
dressed,1


In [25]:
# Store the word frequency data into df3, and calculate the percentage of word occurences.
df3 = df2.loc[df2.index.isin(['phd','python','sas', 'sql', 'tableau', 'r', 'algorithms', 'bachelors', 
                              'hadoop', 'visualization', 'visualizations', 'spark', 'masters', 'aws',
                              'excel', 'agile', 'optimization', 'optimize', 'tensorflow', 'excel', 'nosql', 'scikitlearn', 'nlp'])]
df3['percentage'] = (df3['count'] / len(df.index)) * 100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Visualize the word frequency with bar graphs

In [26]:
import plotly.express as px
fig = px.bar(df3, 
             x=df3.index, 
             y='percentage'
             )
fig.show()

# Modifying dataset

Create the new columns (Boolean type).

Column will have 'True' if it has descriptions and will have 'False' if it does not have a word in descriptions.

In [27]:
df['python'] = df['Job Description'].str.contains('python')
df['phd'] = df['Job Description'].str.contains('phd')
df['sas'] = df['Job Description'].str.contains('sas')
df['sql'] = df['Job Description'].str.contains('sql')
df['tableau'] = df['Job Description'].str.contains('tableau')
df['r'] = df['Job Description'].str.contains('r')
df['algorithms'] = df['Job Description'].str.contains('algorithms')
df['visualization'] = df['Job Description'].str.contains('visualization')
df['spark'] = df['Job Description'].str.contains('spark')
df['masters'] = df['Job Description'].str.contains('masters')
df['aws'] = df['Job Description'].str.contains('aws')
df['excel'] = df['Job Description'].str.contains('excel')
df['agile'] = df['Job Description'].str.contains('agile')
df['optimization'] = df['Job Description'].str.contains('optimization')
df['optimize'] = df['Job Description'].str.contains('optimize')
df['tensorflow'] = df['Job Description'].str.contains('tensorflow')
df['excel'] = df['Job Description'].str.contains('excel')
df['nosql'] = df['Job Description'].str.contains('nosql')
df['hadoop'] = df['Job Description'].str.contains('hadoop')
df['scikitlearn'] = df['Job Description'].str.contains('scikitlearn')
df['nlp'] = df['Job Description'].str.contains('nlp')

# If title includes sr
df['sr'] = df['Job Title'].str.lower().str.contains('sr')

Dataset provides the salary by the range (LOW-HIGH).

Divide that range into two columns, clear some text data for the analysis.


In [28]:
df['low_salary'] = df['Salary Estimate'].str.split('-').str[0]
df['high_salary'] = df['Salary Estimate'].str.split('-').str[1]
df['high_salary'] = df['high_salary'].str.split(" ").str[0]
df['high_salary'] = df['high_salary'].str[:3]

df['Company Name'] = df['Company Name'].str.split('\n').str[0]
df['Location'] = df['Location'].str.split('\n').str[0]

# Create the new columns (State)
df['state'] = df['Location'].str.split(', ').str[1]

Drop some variables that are not needed for the analysis

In [29]:
df.drop(['Job Description', 'Salary Estimate'], axis=1, inplace = True)

If the cell contains the 'True' then convert them into '1' (if it contains 'False' then '0')

In [30]:
df = df.applymap(lambda x: 1 if x == True else x)
df = df.applymap(lambda x: 0 if x == False else x)

# Get Information

Create the new dataframe that 

In [31]:
dfc = df.groupby(['state']).size().to_frame().sort_values([0], ascending = False).reset_index()
dfc.rename(columns={0:'count'}, inplace=True)

In [32]:
import plotly.express as px

dfg = df.groupby(['state']).size().to_frame().sort_values([0], ascending = False).head(10).reset_index()
dfg.rename(columns={0:'count'}, inplace=True)
dfg['percentage'] = dfg['count'] / len(df.index) * 100
fig = px.bar(dfg, x='state', y = 'percentage')

fig.show()

In [33]:
df2 = pd.DataFrame(data=[['CA', df.loc[df['state'] == 'CA', 'high_salary'].median(), df.loc[df['state'] == 'CA', 'low_salary'].median()],
                         ['VA', df.loc[df['state'] == 'VA', 'high_salary'].median(), df.loc[df['state'] == 'VA', 'low_salary'].median()],
                         ['MA', df.loc[df['state'] == 'MA', 'high_salary'].median(), df.loc[df['state'] == 'MA', 'low_salary'].median()],
                         ['NY', df.loc[df['state'] == 'NY', 'high_salary'].median(), df.loc[df['state'] == 'NY', 'low_salary'].median()],
                         ['MD', df.loc[df['state'] == 'MD', 'high_salary'].median(), df.loc[df['state'] == 'MD', 'low_salary'].median()],
                         ['IL', df.loc[df['state'] == 'IL', 'high_salary'].median(), df.loc[df['state'] == 'IL', 'low_salary'].median()],
                         ['DC', df.loc[df['state'] == 'DC', 'high_salary'].median(), df.loc[df['state'] == 'DC', 'low_salary'].median()],
                         ['TX', df.loc[df['state'] == 'TX', 'high_salary'].median(), df.loc[df['state'] == 'TX', 'low_salary'].median()],
                         ['WA', df.loc[df['state'] == 'WA', 'high_salary'].median(), df.loc[df['state'] == 'WA', 'low_salary'].median()],
                         ['OH', df.loc[df['state'] == 'OH', 'high_salary'].median(), df.loc[df['state'] == 'OH', 'low_salary'].median()]],
                   columns=['state','high','low' ])

In [34]:
import plotly.express as px

fig = px.bar(df2, x='state', y = 'high')

fig.show()

In [35]:
import plotly.express as px

fig = px.bar(df2, x='state', y = 'low')

fig.show()

## Salary Information
Glassdoor provides estimated salary information by the range <Min, Max>. 

For estimating the salary, estimated min value and max value are used.

## Salary Information - State

In [36]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Box(y=df['low_salary'], name = 'low range'))
fig.add_trace(go.Box(y=df['high_salary'], name = 'high range'))
fig.update_traces(boxpoints='all', jitter=0)
fig.show()

In [37]:
high_state = pd.to_numeric(df['high_salary']).groupby(df['state'])

mean = high_state.median().reset_index()
mean = mean.sort_values('high_salary', ascending=False)

In [38]:
mean.head()

,state,high_salary
7,DE,331.0
10,IA,198.0
1,AZ,168.0
37,WI,167.0
20,MS,165.0


In [39]:
mean.tail()

,state,high_salary
5,CT,116.0
2,Anne Arundel,112.0
27,OK,111.0
34,UT,109.0
31,SC,107.5


In [40]:
low_state = pd.to_numeric(df['low_salary']).groupby(df['state'])

mean = low_state.median().reset_index()
mean = mean.sort_values('low_salary', ascending=False)

In [41]:
mean

,state,low_salary
7,DE,212.0
10,IA,124.0
37,WI,123.5
1,AZ,112.5
21,NC,110.0
28,OR,109.0
0,AL,105.5
20,MS,101.0
30,RI,100.5
6,DC,100.0


In [42]:
mean.head()

,state,low_salary
7,DE,212.0
10,IA,124.0
37,WI,123.5
1,AZ,112.5
21,NC,110.0


In [43]:
mean.tail()

,state,low_salary
32,TN,75.5
14,LA,75.0
13,KS,75.0
5,CT,70.0
2,Anne Arundel,66.0


## Salary Information - Skills

In [44]:
print(' python {}\n phd {}\n sas {}\n sql {}\n tableau {}\n r {}\n algorithms {}\n'
      ' visualization {}\n spark {}\n masters {}\n aws {}\n excel {}\n agile {}\n optimization {}\n'
      ' optimize {}\n tensorflow {}\n nosql {}\n hadoop {}\n scikitlearn {}\n nlp {}\n'.format(
          df.loc[df['python'] == 1, 'high_salary'].median(),
          df.loc[df['phd'] == 1, 'high_salary'].median(),
          df.loc[df['sas'] == 1, 'high_salary'].median(),
          df.loc[df['sql'] == 1, 'high_salary'].median(), 
          df.loc[df['tableau'] == 1, 'high_salary'].median(),
          df.loc[df['r'] == 1, 'high_salary'].median(),
          df.loc[df['algorithms'] == 1, 'high_salary'].median(),
          df.loc[df['visualization'] == 1, 'high_salary'].median(),
          df.loc[df['spark'] == 1, 'high_salary'].median(),
          df.loc[df['masters'] == 1, 'high_salary'].median(),
          df.loc[df['aws'] == 1, 'high_salary'].median(),
          df.loc[df['excel'] == 1, 'high_salary'].median(),
          df.loc[df['agile'] == 1, 'high_salary'].median(), 
          df.loc[df['optimization'] == 1, 'high_salary'].median(),
          df.loc[df['optimize'] == 1, 'high_salary'].median(),
          df.loc[df['tensorflow'] == 1, 'high_salary'].median(),
          df.loc[df['nosql'] == 1, 'high_salary'].median(),
          df.loc[df['hadoop'] == 1, 'high_salary'].median(), 
          df.loc[df['scikitlearn'] == 1, 'high_salary'].median(),
          df.loc[df['nlp'] == 1, 'high_salary'].median()
          )
      )

 python 132.0
 phd 133.0
 sas 132.0
 sql 132.0
 tableau 141.0
 r 133.0
 algorithms 141.0
 visualization 132.0
 spark 141.0
 masters 132.0
 aws 132.0
 excel 132.0
 agile 132.0
 optimization 132.0
 optimize 132.0
 tensorflow 141.0
 nosql 141.0
 hadoop 141.0
 scikitlearn 132.0
 nlp 146.0



In [45]:
print(' python {}\n phd {}\n sas {}\n sql {}\n tableau {}\n r {}\n algorithms {}\n'
      ' visualization {}\n spark {}\n masters {}\n aws {}\n excel {}\n agile {}\n optimization {}\n'
      ' optimize {}\n tensorflow {}\n nosql {}\n hadoop {}\n scikitlearn {}\n nlp {}\n'.format(
          df.loc[df['python'] == 1, 'low_salary'].median(),
          df.loc[df['phd'] == 1, 'low_salary'].median(),
          df.loc[df['sas'] == 1, 'low_salary'].median(),
          df.loc[df['sql'] == 1, 'low_salary'].median(),
          df.loc[df['tableau'] == 1, 'low_salary'].median(),
          df.loc[df['r'] == 1, 'low_salary'].median(),
          df.loc[df['algorithms'] == 1, 'low_salary'].median(),
          df.loc[df['visualization'] == 1, 'low_salary'].median(),
          df.loc[df['spark'] == 1, 'low_salary'].median(),
          df.loc[df['masters'] == 1, 'low_salary'].median(),
          df.loc[df['aws'] == 1, 'low_salary'].median(),
          df.loc[df['excel'] == 1, 'low_salary'].median(),
          df.loc[df['agile'] == 1, 'low_salary'].median(), 
          df.loc[df['optimization'] == 1, 'low_salary'].median(),
          df.loc[df['optimize'] == 1, 'low_salary'].median(),
          df.loc[df['tensorflow'] == 1, 'low_salary'].median(),
          df.loc[df['nosql'] == 1, 'low_salary'].median(),
          df.loc[df['hadoop'] == 1, 'low_salary'].median(), 
          df.loc[df['scikitlearn'] == 1, 'low_salary'].median(),
          df.loc[df['nlp'] == 1, 'low_salary'].median()
          )
      )

 python 91.0
 phd 92.0
 sas 91.0
 sql 91.0
 tableau 91.0
 r 91.0
 algorithms 92.0
 visualization 91.0
 spark 95.0
 masters 90.0
 aws 91.5
 excel 90.0
 agile 95.0
 optimization 92.0
 optimize 90.0
 tensorflow 99.0
 nosql 95.0
 hadoop 92.0
 scikitlearn 95.0
 nlp 101.0



### Salary Information - SKills - Multiple

In [46]:
g = df.groupby('state')
g.apply(lambda x: x[x['python'] == 1]['high_salary'].median())

state
AL              163.0
AZ              181.5
Anne Arundel      NaN
CA              132.0
CO              116.0
CT              116.0
DC              165.0
DE              331.0
FL              131.0
GA              124.0
IA                NaN
IL              143.5
IN              131.0
KS              131.0
LA                NaN
MA              131.0
MD              131.5
MI              165.0
MN              132.0
MO              131.0
MS                NaN
NC              167.0
NE              136.0
NH              164.5
NJ              150.0
NY              147.0
OH              132.0
OK              135.5
OR              155.0
PA              147.0
RI                NaN
SC                NaN
TN              123.0
TX              141.0
UT              109.0
VA              141.0
WA              151.0
WI              225.0
WV              141.0
dtype: float64

In [47]:
g = df.groupby('python') # Employee can do Python (1), Employee cannot do python (0)
print('and aws {}'.format(g.apply(lambda x: x[x['optimization'] == 1]['high_salary'].median()))) # Employee can do Hadoop if 1

and aws python
0    131.0
1    132.0
dtype: float64
